# HCUP + Python

This is a tutorial on working with Healthcare Cost and Utilization Project (HCUP) datasets using Python and other open-source alternatives to SAS and SPSS, which are the two primary data tools supported by HCUP.


The United States Agency for Healthcare Research and Quality has a variety of large, de-identified hospital patient datasets available via its Healthcare Cost and Utilization Project (aka HCUP). They contain broad information on the diagnoses, duration, and type of treatment per patient for each visit, and slightly more detailed information on the type and volume of charges. Because of this, and because HCUP also makes available a unique identifier that can be used to “follow” a patient within a given state, many physicians, epidemiologists, economists, and other researchers use them as a source for retrospective analysis of outcomes and cost-effectiveness.

That said, the data sets can be large enough to cause some headaches. A single year’s worth of the “core” emergency department visit data for the state of California is about 10 million rows in 152 columns, and arrives from HCUP in a 5.8GB flat (non-delimited) file. What’s more, the number and width of columns supplied varies by the state supplying the data, and even varies by year within a given state.

To aid in parsing the datasets, HCUP provides loading program definitions in SAS and SPSS formats. Unfortunately, not everyone has SAS or SPSS available, and even some who do may have needs best met by other environments or prefer to use open source alternatives. Whatever your particular reasons, if you are interested in working with HCUP data without SAS or SPSS, you’ll need some other way to parse, manipulate, and integrate them.

In my case, I am using the Python programing language (especially the excellent pandas library) to parse HCUP data sets and do preliminary cleanup, then a PostgreSQL database for integration and long-term storage. Much of the Python code I am using is rolled into a package called PyHCUP, which is available on PyPI or simply through pip (pip install PyHCUP).


Reference
- http://bielism.blogspot.com/2013/12/hcup-and-python-pt-i-background.html
- http://bielism.blogspot.com/2013/12/hcup-and-python-pt-4-reading-in-data.html
- http://bielism.blogspot.com/2013/12/hcup-and-python-pt-5-nulls-and-pre.html
- https://pypi.org/project/PyHCUP/

```bash
conda create -n iz_pyhcup --copy -y -q python=2.7 ipykernel pandas numpy
source activate iz_pyhcup
echo "y" |pip install PyHCUP
echo "y" |pip install sqlalchemy
source deactivate

```


## 1. Locate Your HCUP Data (and decompress it, if necessary)

HCUP Data typically arrive on a DVD, and may or may not be compressed in a ZIP archive. Pick a directory on your computer and copy into it your data file. If the data are in a ZIP archive, extract them at this time. You should end up with a file somewhere around five to fifteen times larger (in MB) than the ZIP archive.

The decompressed file should end in a .asc file extension, though the file extension may be hidden from view depending on your operating system settings. Its name should correspond to the data it contains. For example, you are working with New York's 2008 State Inpatient Database CORE data, you should end up with a file called something like NY_SID_2008_CORE.asc. Make a note of its location as we'll need to know that later.


## 2. Locate a SAS Load Program Your HCUP Data
Next, you'll need the appropriate SAS Load Program for your particular HCUP data. Continuing with the above example, we need to find the New York SID 2008 CORE data SAS loading program. HCUP provides SAS loading programs in a few different places on its website, depending on whether you which data you are using. Use the links below to locate the appropriate SAS Load Program for your data. If the links don't work, HCUP has probably changed its website; Google should still be able to help you find them.

NIS: http://www.hcup-us.ahrq.gov/db/nation/nis/nissasloadprog.jsp
KID: http://www.hcup-us.ahrq.gov/db/nation/kid/kidsasloadprog.jsp
NEDS: http://www.hcup-us.ahrq.gov/db/nation/neds/nedssasloadprog.jsp
SID, SEDD, and SASD: http://www.hcup-us.ahrq.gov/sasload/sasload_search.jsp
After you locate and download the appropriate SAS Load Program for your data, make a note of its location as we'll need to know that later.


In [ ]:
!wget https://www.hcup-us.ahrq.gov/db/state/sidc/tools/sasload/NY_SID_2016_CORE.sas
!wget https://www.hcup-us.ahrq.gov/db/state/sidc/tools/sasload/NY_SID_2016_CHGS.sas

!wget https://www.hcup-us.ahrq.gov/db/state/sasdc/tools/sasload/NY_SASD_2016_CORE.sas
!wget https://www.hcup-us.ahrq.gov/db/state/sasdc/tools/sasload/NY_SASD_2016_CHGS.sas

!wget https://www.hcup-us.ahrq.gov/db/state/seddc/tools/sasload/NY_SEDD_2016_CORE.sas
!wget https://www.hcup-us.ahrq.gov/db/state/seddc/tools/sasload/NY_SEDD_2016_CHGS.sas

In [33]:
!mv *.sas ./libs


## 3. Get started

### 3.1 Acquire Metadata

The SAS Load Program files supplied by HCUP are built to tell SAS how to load your data. We will use them instead to tell Python how to load your data.

In [2]:
import pyhcup

In [42]:
#data_path = '/data/workspace/hcup/SIDC_NY_2016/NY_SID_2016_CORE.asc'
data_path = 'CHANGEME'
load_path = './libs/NY_SID_2016_CORE.sas'

In [34]:
#build a pandas DataFrame object from meta data
meta_df = pyhcup.sas.meta_from_sas(load_path)

You can verify that the meta data we have parsed out is reasonable by calling the meta_df DataFrame object we've created. This summary output is for the 2016 NY SID CORE, referenced above.

In [35]:
meta_df

,field,informat,position,format,label,width
0,AGE,N3PF.,1,None,Age in years at admission,3
1,AGEDAY,N3PF.,4,None,Age in days (when age < 1 year),3
2,AGEMONTH,N3PF.,7,None,Age in months (when age < 11 years),3
3,AHOUR,N4PF.,10,Z4.,Admission Hour,4
4,AMONTH,N2PF.,14,None,Admission month,2
5,ANESTH,N3PF.,16,None,Method of anesthesia,3
6,ATYPE,N2PF.,19,None,Admission type,2
7,AWEEKEND,N2PF.,21,None,Admission day is a weekend,2
8,BILLTYPE,$CHAR4.,23,None,"Type of bill, UB-04 coding",4
9,BLOOD,N6PF.,27,None,Pints of blood furnished to the patient,6


The number of entries reflects the number of columns specified in the SAS Load Program. There should generally be either above more than one hundred or fewer than ten columns, depending on the type of HCUP data. For your own interest, you can see the first x rows in a pandas DataFrame by using its .head() method. If there are too many columns, pandas will abridge this from a table-style output to a summary-style output as seen above.

In [36]:
meta_df.head()

,field,informat,position,format,label,width
0,AGE,N3PF.,1,None,Age in years at admission,3
1,AGEDAY,N3PF.,4,None,Age in days (when age < 1 year),3
2,AGEMONTH,N3PF.,7,None,Age in months (when age < 11 years),3
3,AHOUR,N4PF.,10,Z4.,Admission Hour,4
4,AMONTH,N2PF.,14,None,Admission month,2


### 3.2 Use the Metadata to Parse the Data

Now that we have the metadata, we can parse the actual data using the pyhcup.sas.df_from_sas function, with our meta_df as the second our argument. However, given the size of the data sets being read in, you may wish to consider reading in only a subset of rows. Do this by passing an nrows argument as well.

In [43]:
#grab the first 10,000 rows in a DataFrame
df = pyhcup.sas.df_from_sas(data_path, meta_df, nrows=10000)
df

,AGE,AGEDAY,AGEMONTH,AHOUR,AMONTH,ANESTH,ATYPE,AWEEKEND,BILLTYPE,BLOOD,...,PRYEAR5,PRYEAR6,PRYEAR7,PRYEAR8,PRYEAR9,PRYEAR10,PRYEAR11,PRYEAR12,PRYEAR13,PRYEAR14
0,NOT,ICE,: U,se o,f,HCU,P,da,ta c,onstit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,con,dit,ion,s of,t,he,HC,UP,Dat,a Use,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,46,-99,-99,900,-9,20,1,0,0111,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,65,-99,-99,1200,-9,20,5,0,0111,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,45,-99,-99,900,-9,20,1,0,0111,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
5,48,-99,-99,800,-9,20,3,0,0111,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
6,46,-99,-99,1500,-9,0,1,0,0111,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
7,58,-99,-99,500,-9,20,3,0,0111,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
8,49,-99,-99,1100,-9,40,1,0,0111,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
9,48,-99,-99,1300,-9,20,1,0,0111,0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


### 3.3 Nulls and Pre-Analysis

__Replacing HCUP Sentinel Values__

Files from HCUP have several different sentinel values used to mark missing, invalid, or otherwise unavailable data. For a full breakdown, see HCUP's documentation on missing values in http://www.hcup-us.ahrq.gov/db/coding.pdf. In most of my work, the details of this are not vital I just need to mark them as null in the data set.


PyHCUP has a built-in function in its parser module to sweep through an imported HCUP data set and mark all the sentinels as missing values. To use it, pass your imported data set to the `parser.replace_sentinels()` function. This may take some time, depending on the size of the data set you've imported and the speed of your computer.



In [46]:
nulls_replaced = pyhcup.parser.replace_sentinels(df)

Check the AGE column (or whichever column you used .describe() with previously) in the replaced and non-replaced data to see the effects.

In [59]:
df['AGE'].describe() #before

count     10000
unique      105
top           0
freq        891
Name: AGE, dtype: object

In [60]:
nulls_replaced['AGE'].describe() #and after

count     10000
unique      105
top           0
freq        891
Name: AGE, dtype: object

__Characterizing Numeric Data__

Although we are only looked at one column above for brevity, the parser.df_replace_sentinels() function has been applied to every column in the data set. You check describe additional columns by substituting in the column name for AGE in the code we've been using so far. Here is another, looking at the total charges per inpatient stay.

In [49]:
nulls_replaced['TOTCHG'].describe()

count    9.997000e+03
mean     4.971867e+04
std      7.471883e+04
min      1.350000e+02
25%      1.555000e+04
50%      2.842700e+04
75%      5.306600e+04
max      1.305811e+06
Name: TOTCHG, dtype: float64

So out of these 10,000 stays, somebody got out for cheap at less than `$200` and somebody had a rough time with a bill greater than `$1.3 million`.

It is possible to describe multiple columns side by side by first preparing a list of columns, then referring to that instead of passing a single column name. In Python, you can make a list by putting all the desired list items in a shared set of square brackets, with commas between the items. For example, we could get ages, total charges, and length of stay all at once with something like this.

In [50]:
col_list = ['AGE', 'TOTCHG', 'LOS']

In [51]:
nulls_replaced[col_list].describe()

,TOTCHG,LOS
count,9.997000e+03,9998.000000
mean,4.971867e+04,5.822364
std,7.471883e+04,8.256446
min,1.350000e+02,0.000000
25%,1.555000e+04,2.000000
50%,2.842700e+04,3.000000
75%,5.306600e+04,7.000000
max,1.305811e+06,162.000000


__Characterizing Non-Numeric Data__

Averages, extreme values, and percentiles are less helpful for describing things like discharge destinations and diagnoses. For non-numeric data, pandas has a built-in .value_counts() method you can use on columns similar to .describe(). As an example, HCUP SID data typically have available a FEMALE column, where a value of 1 indicates the patient was female and 0 indicates the patient was male.

In [52]:
nulls_replaced['FEMALE'].value_counts()

0.0    5020
1.0    4978
Name: FEMALE, dtype: int64

Unlike .describe(), you cannot directly use .value_counts() simultaneously with multiple columns. However, you can use Python to put together a loop that generates value counts for columns in a list. You can also "nest" this kind of count by grouping, such as getting value counts of primary diagnosis by race or discharge destination by homeless status.



__Knowing What Columns Are Available__

HCUP provide documentation with detailed descriptions on which columns are available for which states in which year, as well as what the contents of each column mean. But for day-to-day work, we can get a list of all the columns in a DataFrame by invoking DataFrame.columns on any of our data sets. The results should be the same right now whether you use it on df or nulls_replaced, since one is derived from the other.



In [54]:
list(nulls_replaced.columns)

['AGE',
 'AGEDAY',
 'AGEMONTH',
 'AHOUR',
 'AMONTH',
 'ANESTH',
 'ATYPE',
 'AWEEKEND',
 'BILLTYPE',
 'BLOOD',
 'BWT',
 'DaysToEvent',
 'DHOUR',
 'DIED',
 'DISP_X',
 'DISPUB04',
 'DISPUNIFORM',
 'DMONTH',
 'DQTR',
 'DRG',
 'DRG_NoPOA',
 'DRGVER',
 'DSHOSPID',
 'DXPOA1',
 'DXPOA2',
 'DXPOA3',
 'DXPOA4',
 'DXPOA5',
 'DXPOA6',
 'DXPOA7',
 'DXPOA8',
 'DXPOA9',
 'DXPOA10',
 'DXPOA11',
 'DXPOA12',
 'DXPOA13',
 'DXPOA14',
 'DXPOA15',
 'DXPOA16',
 'DXPOA17',
 'DXPOA18',
 'DXPOA19',
 'DXPOA20',
 'DXPOA21',
 'DXPOA22',
 'DXPOA23',
 'DXPOA24',
 'DXPOA25',
 'DXVER',
 'E_POA1',
 'E_POA2',
 'E_POA3',
 'E_POA4',
 'E_POA5',
 'E_POA6',
 'E_POA7',
 'E_POA8',
 'E_POA9',
 'FEMALE',
 'HCUP_ED',
 'HCUP_OS',
 'HISPANIC',
 'HISPANIC_X',
 'Homeless',
 'HOSPST',
 'I10_DX_Admitting',
 'I10_DX1',
 'I10_DX2',
 'I10_DX3',
 'I10_DX4',
 'I10_DX5',
 'I10_DX6',
 'I10_DX7',
 'I10_DX8',
 'I10_DX9',
 'I10_DX10',
 'I10_DX11',
 'I10_DX12',
 'I10_DX13',
 'I10_DX14',
 'I10_DX15',
 'I10_DX16',
 'I10_DX17',
 'I10_DX18',
 'I10_DX